In [ ]:
from google.colab import files  # Only works in Google Colab

uploaded = files.upload()

# Access the file
for filename in uploaded.keys():
    print(f"Uploaded file: {filename}")

Saving june3.csv to june3 (1).csv
Uploaded file: june3 (1).csv


In [ ]:
import pandas as pd
df = pd.read_excel(filename) if filename.endswith('.xlsx') else pd.read_csv(filename)
print("Column names:")
print(df.columns.tolist())

Column names:
['RecordingTime [ms]', 'Time of Day [h:m:s:ms]', 'Trial', 'Stimulus', 'Export Start Trial Time [ms]', 'Export End Trial Time [ms]', 'Participant', 'Color', 'Tracking Ratio [%]', 'Category Group', 'Category Binocular', 'Index Binocular', 'Pupil Size Right X [px]', 'Pupil Size Right Y [px]', 'Pupil Diameter Right [mm]', 'Pupil Size Left X [px]', 'Pupil Size Left Y [px]', 'Pupil Diameter Left [mm]', 'Point of Regard Binocular X [px]', 'Point of Regard Binocular Y [px]', 'Point of Regard Right X [px]', 'Point of Regard Right Y [px]', 'Point of Regard Left X [px]', 'Point of Regard Left Y [px]', 'AOI Name Binocular', 'Gaze Vector Right X', 'Gaze Vector Right Y', 'Gaze Vector Right Z', 'Gaze Vector Left X', 'Gaze Vector Left Y', 'Gaze Vector Left Z', 'Video Time [h:m:s:ms]', 'Annotation Name', 'Annotation Description', 'Annotation Tags', 'Mouse Position X [px]', 'Mouse Position Y [px]', 'Scroll Direction X', 'Scroll Direction Y', 'Content']


/tmp/ipython-input-50-2196398907.py:2: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_excel(filename) if filename.endswith('.xlsx') else pd.read_csv(filename)


In [ ]:
columns_to_drop = [
    "Video Time [h:m:s:ms]", "Annotation Name", "Annotation Description", "Annotation Tags",
    "Mouse Position X [px]", "Mouse Position Y [px]",
    "Scroll Direction zX", "Scroll Direction Y", "Content","Stimulus","Export Start Trial Time [ms]","AOI Name Binocular",
]
df.drop(columns=columns_to_drop, inplace=True, errors="ignore")

In [ ]:
#Convert the relevant columns to numeric data types
#This helps ensure that any non-numeric entries are turned into NaN so you can handle them.
columns_to_convert = [
    "Pupil Size Right X [px]", "Pupil Size Right Y [px]", "Pupil Diameter Right [mm]",
    "Pupil Size Left X [px]", "Pupil Size Left Y [px]", "Pupil Diameter Left [mm]",
    "Point of Regard Binocular X [px]", "Point of Regard Binocular Y [px]",
    "Point of Regard Right X [px]", "Point of Regard Right Y [px]",
    "Point of Regard Left X [px]", "Point of Regard Left Y [px]",
    "Gaze Vector Right X", "Gaze Vector Right Y", "Gaze Vector Right Z",
    "Gaze Vector Left X", "Gaze Vector Left Y", "Gaze Vector Left Z"
]

df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors="coerce")

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
from google.colab import files
def impute(series, short=4, med=20):
    s = series.copy()
    mask = s.isna()
    groups = (mask != mask.shift()).cumsum()
    for _, block in s.groupby(groups):
        if not block.isna().all(): continue
        start = block.index[0] - 1
        end = block.index[-1] + 1
        length = end - start - 1
        if start < 0 or end >= len(s): continue
        b, a = s.iat[start], s.iat[end]
        # Short gap: linear
        if length <= short:
            for i in range(1, length+1):
                ratio = i/(length+1)
                s.iat[start+i] = b + ratio*(a-b)
        # Medium gap: spline if enough neighbors
        elif length <= med:
            idxs = [i for i in range(start-3, end+4) if 0<=i<len(s) and not pd.isna(s.iat[i])]
            if len(idxs) >= 4:
                xs, ys = np.array(idxs), s.iloc[idxs].values
                cs = CubicSpline(xs, ys)
                for i in range(1, length+1): s.iat[start+i] = cs(start+i)
            else:
                for i in range(1, length+1):
                    ratio = i/(length+1)
                    s.iat[start+i] = b + ratio*(a-b)
        # Long gaps: leave NaN
    return s

for col in ['Pupil Diameter Left [mm]', 'Pupil Diameter Right [mm]',
            'Point of Regard Binocular X [px]', 'Point of Regard Binocular Y [px]',
            'Tracking Ratio [%]']:
    if col in df.columns:
        df[col] =impute(df[col])

In [ ]:
import numpy as np

# Calculate pupil diameter avg (temporary)
pupil_diameter_avg = df[['Pupil Diameter Left [mm]', 'Pupil Diameter Right [mm]']].mean(axis=1)

# Calculate time in seconds (temporary)
recording_time_s = df['RecordingTime [ms]'] / 1000.0

# Calculate Point of Regard velocity (temporary)
POR_X_diff = df['Point of Regard Binocular X [px]'].diff()
POR_Y_diff = df['Point of Regard Binocular Y [px]'].diff()
time_diff = recording_time_s.diff()
POR_velocity = np.sqrt(POR_X_diff**2 + POR_Y_diff**2) / time_diff

# Initialize the category
df['Category Binocular'] = 'uncategorized'

# Blink detection: Pupil size drops or tracking ratio = 0
blink_mask = (
    (pupil_diameter_avg < 1.5) |
    (df['Tracking Ratio [%]'] < 10 )
)
df.loc[blink_mask, 'Category Binocular'] = 'blink'

# Saccade detection: Higher velocity threshold, short duration
saccade_mask = (
    (POR_velocity > 1200) &  # Increased from 100
    (time_diff < 0.15) &
    (~blink_mask)
)
df.loc[saccade_mask, 'Category Binocular'] = 'saccade'

# Visual Intake: More tolerant velocity and tracking ratio
visual_intake_mask = (
    (POR_velocity < 1200) &  # Increased from 100
    (df['Tracking Ratio [%]'] > 5) &  # Lowered from 10
    (~blink_mask) &
    (~saccade_mask)
)
df.loc[visual_intake_mask, 'Category Binocular'] = 'visual intake'

# Output
print(df[['RecordingTime [ms]', 'Category Binocular']].head(30))


    RecordingTime [ms] Category Binocular
0            5173630.8      uncategorized
1            5173647.5      visual intake
2            5173664.1      visual intake
3            5173680.7      visual intake
4            5173697.3      visual intake
5            5173714.0            saccade
6            5173730.7            saccade
7            5173747.2      visual intake
8            5173764.0      visual intake
9            5173780.5      visual intake
10           5173797.2      visual intake
11           5173813.9      visual intake
12           5173830.5      visual intake
13           5173847.2      visual intake
14           5173863.8      visual intake
15           5173880.4      visual intake
16           5173897.1      visual intake
17           5173913.7      visual intake
18           5173930.3      visual intake
19           5173947.0      visual intake
20           5173963.6      visual intake
21           5173980.3      visual intake
22           5173996.9            

In [ ]:
# Count the number of Visual Intake entries
visual_intake_count = (df['Category Binocular'] == 'visual intake').sum()

print(f"Number of Visual Intake events: {visual_intake_count}")


Number of Visual Intake events: 75772


In [ ]:


# Show all columns
pd.set_option('display.max_columns', None)

# Show all rows (optional)
# pd.set_option('display.max_rows', None)

# Now this will show all columns
print(df.head())


   RecordingTime [ms] Time of Day [h:m:s:ms]     Trial  \
0           5173630.8           11:22:25:297  Trial001   
1           5173647.5           11:22:25:314  Trial001   
2           5173664.1           11:22:25:330  Trial001   
3           5173680.7           11:22:25:347  Trial001   
4           5173697.3           11:22:25:363  Trial001   

   Export End Trial Time [ms] Participant  Color  Tracking Ratio [%]  \
0                    181364.6   OH21_AM_1  Coral               100.0   
1                    181364.6   OH21_AM_1  Coral               100.0   
2                    181364.6   OH21_AM_1  Coral               100.0   
3                    181364.6   OH21_AM_1  Coral               100.0   
4                    181364.6   OH21_AM_1  Coral               100.0   

  Category Group Category Binocular Index Binocular  Pupil Size Right X [px]  \
0            Eye      uncategorized               1                     62.0   
1            Eye      visual intake               1       

In [ ]:
import pandas as pd
# Column mapping
column_mapping = {
    'timestamp': 'Time of Day [h:m:s:ms]',  # Human-readable time
    'RecordingTime [ms]': 'RecordingTime [ms]',
    'gaze_x': 'Gaze Vector Right X',
    'gaze_y': 'Gaze Vector Right Y',
    'left_pupil_x': 'Pupil Size Left X [px]',
    'left_pupil_y': 'Pupil Size Left Y [px]',
    'left_pupil_diameter': 'Pupil Diameter Left [mm]',
    'right_pupil_x': 'Pupil Size Right X [px]',
    'right_pupil_y': 'Pupil Size Right Y [px]',
    'right_pupil_diameter': 'Pupil Diameter Right [mm]',
    'PoR_binocular_x': 'Point of Regard Binocular X [px]',
    'PoR_binocular_y': 'Point of Regard Binocular Y [px]',
    'Point of Regard Right X': 'Point of Regard Right X [px]',
    'Point of Regard Right Y': 'Point of Regard Right Y [px]',
    'Point of Regard Left X': 'Point of Regard Left X [px]',
    'Point of Regard Left Y': 'Point of Regard Left Y [px]',
    'Category Binocular': 'Category Binocular',
    'Index Binocular': 'Index Binocular',
}

# Final column order
final_columns = ['timestamp', 'RecordingTime [ms]', 'gaze_x', 'gaze_y','blink',
                 'saccade_velocity', 'fixation', 'pupil_size',
                 'left_pupil_x', 'left_pupil_y', 'left_pupil_diameter',
                 'right_pupil_x', 'right_pupil_y', 'right_pupil_diameter',
                 'PoR_binocular_x', 'PoR_binocular_y',
                 'Point of Regard Right X', 'Point of Regard Right Y',
                 'Point of Regard Left X', 'Point of Regard Left Y',
                 'Category Binocular', 'Index Binocular']

# Select and rename required columns
required_columns = list(column_mapping.values())
df_cleaned = df[required_columns].copy()
df_cleaned.rename(columns={v: k for k, v in column_mapping.items()}, inplace=True)

# Add missing columns
missing_columns = ['blink', 'saccade_velocity', 'fixation', 'pupil_size']
for col in missing_columns:
    df_cleaned[col] = None  # You can calculate later

# Mark blink rows
df_cleaned['blink'] = df_cleaned['Category Binocular'].apply(lambda x: 1 if str(x).lower() == 'blink' else 0)

# Reorder columns
df_cleaned = df_cleaned[final_columns]




In [ ]:
print(df_cleaned.head())

      timestamp  RecordingTime [ms]  gaze_x  gaze_y  blink saccade_velocity  \
0  11:22:25:297           5173630.8     0.0     0.0      0             None   
1  11:22:25:314           5173647.5     0.0     0.0      0             None   
2  11:22:25:330           5173664.1     0.0     0.0      0             None   
3  11:22:25:347           5173680.7     0.0     0.0      0             None   
4  11:22:25:363           5173697.3     0.0     0.0      0             None   

  fixation pupil_size  left_pupil_x  left_pupil_y  left_pupil_diameter  \
0     None       None          76.0          48.0                  2.9   
1     None       None          76.0          50.0                  2.9   
2     None       None          76.0          52.0                  2.9   
3     None       None          76.0          48.0                  2.9   
4     None       None          76.0          52.0                  2.9   

   right_pupil_x  right_pupil_y  right_pupil_diameter  PoR_binocular_x  \
0     

In [ ]:
import pandas as pd
import numpy as np

# Calculate saccade velocity using temporary variables
time_diff_s = df_cleaned['RecordingTime [ms]'].diff() / 1000.0
gaze_x_diff = df_cleaned['gaze_x'].diff()
gaze_y_diff = df_cleaned['gaze_y'].diff()

saccade_velocity = np.sqrt(gaze_x_diff**2 + gaze_y_diff**2) / time_diff_s
saccade_velocity = saccade_velocity.replace([np.inf, -np.inf], np.nan).fillna(0)
df_cleaned['saccade_velocity'] = saccade_velocity

# Identify fixations: velocity < 30 pixels/second
df_cleaned['fixation'] = df_cleaned.apply(
    lambda row: 1 if (row['saccade_velocity'] < 50 and row['saccade_velocity'] > 0  and row['blink'] ==0) else 0,
    axis=1
)


# Calculate pupil size as average of left and right diameters
df_cleaned['pupil_size'] = df_cleaned[['left_pupil_diameter', 'right_pupil_diameter']].mean(axis=1)

# Final cleaned dataframe
df_cleaned = df_cleaned[final_columns]

# Save or display cleaned file
output_path = '/content/cleaned_file.csv'  # Change path as needed
df_cleaned.to_csv(output_path, index=False)

print(f'Cleaned file saved at: {output_path}')
print(df_cleaned.head())


Cleaned file saved at: /content/cleaned_file.csv
      timestamp  RecordingTime [ms]  gaze_x  gaze_y  blink  saccade_velocity  \
0  11:22:25:297           5173630.8     0.0     0.0      0               0.0   
1  11:22:25:314           5173647.5     0.0     0.0      0               0.0   
2  11:22:25:330           5173664.1     0.0     0.0      0               0.0   
3  11:22:25:347           5173680.7     0.0     0.0      0               0.0   
4  11:22:25:363           5173697.3     0.0     0.0      0               0.0   

   fixation  pupil_size  left_pupil_x  left_pupil_y  left_pupil_diameter  \
0         0        2.60          76.0          48.0                  2.9   
1         0        2.65          76.0          50.0                  2.9   
2         0        2.60          76.0          52.0                  2.9   
3         0        2.60          76.0          48.0                  2.9   
4         0        2.60          76.0          52.0                  2.9   

   right_pupi

In [ ]:
fixation_count = df_cleaned['blink'].sum()
print(f"Number of fixations: {fixation_count}")


Number of fixations: 21


In [ ]:
import pandas as pd

# Make sure data is sorted by time
df_cleaned = df_cleaned.sort_values('RecordingTime [ms]').reset_index(drop=True)

# Step 1: Clean possible spaces in Category Binocular (just in case)
df_cleaned['Category Binocular'] = df_cleaned['Category Binocular'].str.strip()

# Step 2: Mark groups of continuous blinks using the Blink column
df_cleaned['group_change'] = (df_cleaned['blink'] != df_cleaned['blink'].shift()).cumsum()

# Step 3: Filter only Blink = 1 groups
blink_groups = df_cleaned[df_cleaned['blink'] == 1].groupby('group_change').agg({
    'RecordingTime [ms]': ['first', 'last'],
    'blink': 'count'
}).reset_index()

blink_groups.columns = ['blink_group', 'start_time', 'end_time', 'blink_length']

# Step 4: Calculate blink duration in ms
blink_groups['blink_duration'] = blink_groups['end_time'] - blink_groups['start_time']

# Step 5: Classify short and long blinks (threshold 150 ms)
short_blinks = blink_groups[blink_groups['blink_duration'] < 250]
long_blinks = blink_groups[blink_groups['blink_duration'] >= 250]

print(f"Total short blinks: {len(short_blinks)}")
print(f"Total long blinks: {len(long_blinks)}")

# Step 6: Mark short and long blink rows in the main DataFrame
short_blink_groups = short_blinks['blink_group'].values
long_blink_groups = long_blinks['blink_group'].values

df_cleaned['is_short_blink'] = df_cleaned['group_change'].isin(short_blink_groups).astype(int)
df_cleaned['is_long_blink'] = df_cleaned['group_change'].isin(long_blink_groups).astype(int)

# If you want to see the blink events:
# print(short_blinks)
# print(long_blinks)

# Optional: Save blink data
# short_blinks.to_excel('short_blinks.xlsx', index=False)
# long_blinks.to_excel('long_blinks.xlsx', index=False)


Total short blinks: 16
Total long blinks: 0


In [ ]:
import pandas as pd
import numpy as np

window_size = 10000

if len(df_cleaned) < window_size:
    print(f"❌ Data too small for window size {window_size}")
else:
    # Step 1: Create binary indicators
    df_cleaned['is_uncategorized'] = (df_cleaned['Category Binocular'].str.lower() == 'uncategorized').astype(int)
    df_cleaned['is_long_blink'] = df_cleaned['is_long_blink'].fillna(0).astype(int)

    # Step 2: Compute rolling sums
    long_blinks_rolling = df_cleaned['is_long_blink'].rolling(window=window_size).sum()
    uncategorized_rolling = df_cleaned['is_uncategorized'].rolling(window=window_size).sum()

    # Step 3: Build a combined score: prioritize uncategorized minimization, but penalize long blinks
    # For example: 1 long blink = 5 uncategorized points
    penalty_weight = 5  # You can tweak this
    final_score = (long_blinks_rolling * penalty_weight) + uncategorized_rolling

    # Step 4: Find the window with the minimum score
    best_window_end = final_score.dropna().idxmin()
    best_window_start = best_window_end - window_size + 1

    best_segment = df_cleaned.iloc[best_window_start:best_window_end + 1]

    print(f"✅ Best 10,000-row window found!")
    print(f"🔹 Start index: {best_window_start}")
    print(f"🔹 End index: {best_window_end}")
    print(f"🔹 Total long blinks: {best_segment['is_long_blink'].sum()}")
    print(f"🔹 Total uncategorized: {best_segment['is_uncategorized'].sum()}")

    # Optional: Save best window
    best_segment.to_csv("best_10000_segment.csv", index=False)
    print("✅ Saved as 'best_10000_segment.csv'")

✅ Best 10,000-row window found!
🔹 Start index: 0
🔹 End index: 9999
🔹 Total long blinks: 0
🔹 Total uncategorized: 0
✅ Saved as 'best_10000_segment.csv'


In [ ]:
# Strip spaces and lowercase all column names
df.columns = df.columns.str.strip().str.lower()

# Check new column names
print(df.columns)


Index(['recordingtime [ms]', 'time of day [h:m:s:ms]', 'trial',
       'export end trial time [ms]', 'participant', 'color',
       'tracking ratio [%]', 'category group', 'category binocular',
       'index binocular', 'pupil size right x [px]', 'pupil size right y [px]',
       'pupil diameter right [mm]', 'pupil size left x [px]',
       'pupil size left y [px]', 'pupil diameter left [mm]',
       'point of regard binocular x [px]', 'point of regard binocular y [px]',
       'point of regard right x [px]', 'point of regard right y [px]',
       'point of regard left x [px]', 'point of regard left y [px]',
       'gaze vector right x', 'gaze vector right y', 'gaze vector right z',
       'gaze vector left x', 'gaze vector left y', 'gaze vector left z',
       'scroll direction x'],
      dtype='object')


In [ ]:
from google.colab import files

# Save the best segment to a CSV file
best_segment_path = 'best_segment_cleaned.csv'
best_segment.to_csv(best_segment_path, index=False)

# Download the file
files.download(best_segment_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>